In [1]:
import pandas as pd
import plotly.express as px
import os

In [2]:
data_dir = "data/kaggle_dl"

We have three building

In [3]:
os.listdir(os.path.join(data_dir, 'RBHU-2025-01','RBHU'))

['B205', 'B106', 'B201']

# Metadata Exploration

In total we have almost 6K sensors in all buildings

In [ ]:
metadata = pd.read_parquet(os.path.join(data_dir, "metadata.parquet"))
print(metadata.shape)

In [ ]:
metadata['description_without_room'] = metadata.apply(lambda row: row['description'].replace(str(row['room']), '') if pd.notnull(row['room']) else row['description'], axis=1)
metadata['short_description'] = metadata['description_without_room'].apply(lambda x: ' '.join(x.split(' ')[:2]))
metadata.head()

In [ ]:
metadata[metadata['channel'] == 'AM01']['short_description'].value_counts()

In [ ]:
channel_counts = metadata['channel'].value_counts()
selected = channel_counts[channel_counts > 20]
most_pop_short_desc_info = {}
missing_room_ratio = {}
for channel_id in selected.index:
    top_short_counts = metadata[metadata['channel'] == channel_id]['short_description'].value_counts()
    top_short_counts = top_short_counts / top_short_counts.sum()
    most_pop_short_desc_info[channel_id] = top_short_counts.index[0] + f" ({top_short_counts.iloc[0]*100:.1f}%)"
    missing_room_ratio[channel_id] = metadata[metadata['channel'] == channel_id]['room'].isna().mean()
channel_info_df = pd.DataFrame({'most_popular_short_description': most_pop_short_desc_info, 'missing_room_ratio': missing_room_ratio})
selected = pd.DataFrame(selected).join(channel_info_df)
selected.columns = ['sensor_count', 'most_popular_short_description', 'missing_room_ratio']
print(selected.head())
selected.to_csv('channel_groups_by_most_common_short_description.csv', index=True)
px.bar(selected, title="Channels with more than 30 sensors", text='most_popular_short_description', color='missing_room_ratio', log_y=True).show()

In [ ]:
from feature_groups import get_room_temperatures

selected_fc_meta = get_room_temperatures(metadata, class_id='FC')
len(selected_fc_meta)

In [ ]:
selected_fc_meta['object_id'].nunique(), selected_fc_meta['room'].nunique()

In [ ]:
selected_rc_meta = get_room_temperatures(metadata, class_id='RC')
len(selected_rc_meta)

In [ ]:
selected_rc_meta['room'].nunique()

In [ ]:
channel_counts = metadata['channel'].value_counts()

In [ ]:
channel_counts['AC21'], channel_counts['VT03_2'], channel_counts['VT03_1']

In [ ]:
active_sp = metadata[metadata['channel']=='VT03_2']
active_sp.isnull().mean().sort_values(ascending=False).head(10)

In [ ]:
active_sp.nlargest(150, 'bim_room_area').plot.bar(x='room', y='bim_room_area')

In [ ]:
metadata['object_id'].value_counts().value_counts()

In [ ]:
'building' in metadata.columns

Based on this map: https://www.homeofbosch.com/en/projects/campus-with-rooftop-gardens/
- B201 is office building + canteen (part of new innovation campus)
- B205 is energy related building (part of new innovation campus)
- B106 is on the other side (part of old campus)
- layout of the innovation campus buildings: https://scitope.com/cogmob22/?page_id=26

So I think that the task is to predict the consumption of B201. 

In [ ]:
metadata['building'].value_counts()

In [ ]:
metadata.head()

In [ ]:
metadata.columns

Information about location within floor of a building is rarely available

In [ ]:
metadata.isnull().mean().sort_values(ascending=False).plot.bar(figsize=(15,3))

bde_group is a grouping of sensors

In [ ]:
metadata['bde_group'].value_counts().plot.bar(figsize=(15,3))

In [ ]:
metadata[metadata['bde_group']!='Other']['building'].value_counts()

In [ ]:
metadata[metadata['bde_group']=='Other']['building'].value_counts()

In [ ]:
class_counts = metadata['class_id'].value_counts()
print(class_counts)

In [ ]:
sensors_per_class = {}
for class_id, _ in class_counts.items():
    class_metadata = metadata[metadata['class_id'] == class_id]
    sensors_per_class[class_id] = class_metadata

In [ ]:
external_measurements = [
    'B106WS01.AM50',  # light direction
    'B106WS01.AM51',  # light intensity
    'B106WS01.AM52',  # air pressure
    'B106WS01.AM53',  # humidity
    'B106WS01.AM54',  # external temperature
    'B106WS01.AM54_1',  # outdoor air 30min (AUL-TEMP)
    'B106WS01.AM54_2',  # outdoor air 24h (AUL-TEMP)
    'B106WS01.AM55',  # precipitation
    'B106WS01.AM56',  # wind direction
    'B106WS01.AM57',  # wind speed
]

Sensors of any type
- first few sensors are external measurements on B106
- then comes a lot of temperature sensors and fan capacity for multiple buildings
- resolution is rarely high (ususally 5-10 min)

In [ ]:
sensors_per_class[19.0].head(20)

#tmp = corr_with_all_meta_df[corr_with_all_meta_df['description'].str.upper().str.contains('SUPPLY-AIR')]#['floor'].value_counts()#.isnull().mean().sort_values(ascending=False).plot.bar(figsize=(15,3))
#tmp = corr_with_all_meta_df[corr_with_all_meta_df['description'].str.upper().str.contains('POWER')]
tmp = corr_with_all_meta_df[corr_with_all_meta_df['description'].str.upper().str.contains('VOLUME')]
#tmp = corr_with_all_meta_df[corr_with_all_meta_df['description'].str.upper().str.contains('PRESSURE')] # low corr
tmp.groupby(['conversion_index','dimension_index'])['abs_correlation'].agg(['mean', 'min', 'max', 'std', 'count']).sort_values('mean', ascending=False)#.head(10)

metadata[(metadata['description'].str.upper().str.contains('VOLUME'))]

tmp.groupby('conversion_index')['dimension_index'].nunique().sort_values(ascending=False)

In [ ]:
sensors_per_class[19.0]['dimension_text'].value_counts().head(10)

In [ ]:
sensors_per_class[19.0]['resolution'].value_counts()

In [ ]:
sensors_per_class[19.0]['building'].value_counts()

Valves, fans, bypass
- these sensors mostly measure some intensity (%)
- mostly high-resolution sensors (120 sec)

In [ ]:
sensors_per_class[22.0].head(5)

In [ ]:
sensors_per_class[22.0]['dimension_text'].value_counts().head(10)

In [ ]:
sensors_per_class[22.0]['resolution'].value_counts()

In [ ]:
sensors_per_class[22.0]['building'].value_counts()

These are defaults settings?
- setpoints, preset room temperature

In [ ]:
sensors_per_class[744.0].head(5)

In [ ]:
sensors_per_class[744.0]['dimension_text'].value_counts().head(10)

In [ ]:
sensors_per_class[744.0]['resolution'].value_counts().head(10)

Units with detailed position within building
- we have room area, position, type etc.
- only one dimension (ml/h), these could be the AC or heating units?

In [ ]:
sensors_per_class[143.0].head(5)

In [ ]:
sensors_per_class[143.0]['dimension_text'].value_counts().head(10)

In [ ]:
sensors_per_class[143.0]['resolution'].value_counts().head(10)

In [ ]:
sensors_per_class[143.0]['building'].value_counts()

Quite many units are available for B205
- unfortunately for this building we don't have detailed room information

In [ ]:
sensors_per_class[140.0].head(5)

In [ ]:
sensors_per_class[140.0]['building'].value_counts()

In [ ]:
metadata[(metadata['building']=='B205') & (metadata['class_id']==140.0)]

Some sensors for maintenance or technical rooms?

In [ ]:
sensors_per_class[1324.0]

Technical infor only for B205

In [ ]:
sensors_per_class[109.0]

Examining sensor types

In [ ]:
target = "B205WC000.AM02" # a return temperature chilled water (cooling load of the building)
variables = [
    "B205WC000.AM01",  # a supply temperature chilled water
    "B106WS01.AM54",  # an external temperature
]
#both of these are high resolution (2min) sensors

selected_sensors = metadata[metadata['object_id'].isin(variables + [target])]
selected_sensors.T

Just a few sensors are from target building B205

In [ ]:
px.bar(metadata['building'].value_counts(), title="Number of sensors per building", log_y=True).show()

Multiple external measurements are available about weather

In [ ]:
metadata['external'] = metadata['object_id'].isin(external_measurements)
metadata['external'].value_counts()

# Time Series Exploration

In [ ]:
metadata[metadata['description'].str.upper().str.contains('RETURN TEMPERATURE CHILLED')][['object_id', 'device', 'description', 'file']]

In [ ]:
metadata[metadata['object_id']=='B201FC058_4.AM01'].T

In [ ]:
metadata[metadata['room']=='201.B.040']#.isnull().mean()

For a large labore a lot of measurements are missing

In [ ]:
metadata[metadata['room']=='201.B.040']#.isnull().mean()
#['object_id'].tolist()

In [ ]:
show_these = [
    'B205WC000.AM02',#target (WC000)
    #'B205WC001.AM02',#target - different device (WC001) - due to high correlation it was not provided
    'B205WC000.AM01',#a supply temp chilled water (WC000)
    #'B205WC001.AM01',#a supply temp chilled water - different device (WC001)
    #'B205WC003.AM01',#a supply temp chilled water - 3 buildings only
    #'B201FC058_4.AM01',#room temp
    #'B201FC063_1.VS01_1',
    #'B201FC268_2.VS01_1',
    #'B201RC058.AM21',#labore co2
    #'B201RC058.AM01',#labor rc temp
    #'B201FC058_1.DC21', #labor fc 1 on off
    #'B201FC058_1.VT01_1', #labor fc 1 active heating sp
    #'B201FC058_1.VT02_1', #labor fc 1 active cooling sp
    #'B201FC058_1.VT03_2', #labor fc 1 active sp
    #'B201FC058_1.AM01',#labor fc temp 1 - missing for most of the months
    #'B201FC058_8.AM01',#labor fc temp 8 - missing for most of the months
    #'B201FC058_9.AM01',#labor fc temp 9 - missing for most of the months
    'B106WS01.AM54',  # external temperature
    #'B106WS01.AM55',  # precipitation
    #'B106WS01.AM53',  # humidity
    "B201FC001_1.DC21",
    "B201FC001_1.AM01",
    "B201FC001_1.VT01_1",
    "B201FC001_1.VT02_1",
    "B201FC001_1.VT03_2",
    #"B201FC001_2.AM01",
    #"B201FC001_2.VT01_1",
    #"B201FC001_2.VT02_1",
    #"B201FC001_2.VT03_2",
]
month_ids = [
    #'RBHU-2024-07',
    #'RBHU-2024-08',
    #'RBHU-2024-09',
    #'RBHU-2024-10',
    #'RBHU-2024-11',
    #'RBHU-2024-12', 
    #'RBHU-2025-01',
    'RBHU-2025-02',
    'RBHU-2025-03',
    'RBHU-2025-04',
    'RBHU-2025-05',
    'RBHU-2025-06',
]

In [ ]:
cooler_valves = metadata[metadata['channel']=='AC21']
cooler_valves.shape

In [ ]:
cooler_valves.isnull().mean().sort_values(ascending=False)

In [ ]:
cooler_valves['floor'].value_counts()

In [ ]:
cooler_valves['room'].value_counts()

#show_these = cooler_valves['object_id'].tolist()
show_these = cooler_valves[cooler_valves['room'].isnull()]['object_id'].tolist()
month_ids = [
    'RBHU-2025-04',
    'RBHU-2025-05',
    'RBHU-2025-06',
]
len(show_these)

show_these += [
    'B201FC112_1.AM01',
    'B201FC112_1.DC21',
    'B201FC112_1.VS01_1',
    'B201FC112_1.VT03_1',
    'B201FC112_1.VT03_2'
 ]

In [ ]:
selected_metadata = metadata[metadata['object_id'].isin(show_these)].drop_duplicates(subset=['file'])
ts_parts = []
for month_id in month_ids:
    for _, row in selected_metadata.iterrows():
        if row['file'] is None:
            continue
        file_path = os.path.join(data_dir, month_id, row['file'])
        if not os.path.exists(file_path):
            print(f"File {file_path} does not exist, skipping...")
            continue
        print(f"Loading {file_path}...")
        ts_part = pd.read_parquet(file_path)
        ts_parts.append((row['object_id'], ts_part))

Without interpolation the data is quite sparse !!!

In [ ]:
resampled_series = []
for sensor, ts in ts_parts:
    print(f"Resampling sensor {sensor}...")
    resampled = ts.set_index('time').resample('10min').mean().interpolate().reset_index()
    #resampled = ts.set_index('time').resample('10min').mean().reset_index()
    resampled['sensor'] = sensor
    resampled_series.append(resampled)
resampled_df = pd.concat(resampled_series)

In [ ]:
resampled_pivot = resampled_df.pivot(index='time', columns='sensor', values='data').reset_index()
resampled_pivot.head()

In [ ]:
resampled_pivot.isnull().mean()

For a given room: room temperature (conversion_index=118.0) and status_step ventillation (conversion_index=303.0) are the less sparse features. they are still sparse!

In [ ]:
metadata[metadata['room']=='201.A.134']#['object_id'].tolist()

Room temperature control observations:
- status ventillation has effect for room temperature
- for setpoints I cannot see any effect
   - maybe: when temp is above setpoint, ventillation is turned on to cool down the room
   - but it does not follow this logic all the time, why?

Feature engineering ideas:
- craft features about ventillation status changes within the building
   - set is 2 when cooling is needed (in case of higher room temperature)
   - 0 when cooling is turned off (room temp below setpoint)
- weighted for room area?
- with respect to room energy category?

In [ ]:
resampled_visu = resampled_df#[resampled_df['time'] < '2025-01-31']
px.line(resampled_visu, x='time', y='data', color='sensor').show()

# Correlation analysis

corr_df = pd.read_csv('b205_sensor_correlations.csv')
corr_df.shape

corr_df.head()

shifted_corr_df = pd.read_csv('b205_sensor_correlations_shift-180min.csv')
#shifted_corr_df = pd.read_csv('5months_2buildings_corrs.csv')
print(shifted_corr_df.shape)
shifted_corr_df.head()

In [ ]:
shifted_corr_df = pd.read_csv('correlation_analysis_shifted-180/correlation_summary_across_splits.csv')
shifted_corr_df = shifted_corr_df[['sensor_id', 'mean_correlation', 'mean_abs_correlation']]
shifted_corr_df.rename(columns={
    'mean_correlation': 'correlation',
    'mean_abs_correlation': 'abs_correlation'
}, inplace=True)
print(shifted_corr_df.shape)
shifted_corr_df.head()

The 180 minutes shift does not change the landscape much. The same set of sensors are correlated with the target sensor in both cases.

merged_df = corr_df.merge(shifted_corr_df, on='sensor_id', suffixes=('_no_shift', '_shifted_180min'))
merged_df[['correlation_no_shift', 'correlation_shifted_180min']].corr(method='spearman')

In [ ]:
corr_with_meta_df = shifted_corr_df.merge(metadata[['object_id', 'description','dimension_text', 'floor']], left_on='sensor_id', right_on='object_id', how='left').drop(columns=['object_id']).sort_values(by='abs_correlation', ascending=False)

In [ ]:
corr_with_meta_df.head(30)

In [ ]:
threshold = 0.4
high_abs_corr_df = corr_with_meta_df[ corr_with_meta_df['abs_correlation'] > threshold ]
for _, row in high_abs_corr_df.iterrows():
    print(f"'{row['sensor_id']}',# {row['description']}")

ml_model_weights_fp = 'feature_selection_outs/lasso_feature_weights.csv'
ml_model_weights_df = pd.read_csv(ml_model_weights_fp)
ml_model_weights_df.sort_values('abs_weight', ascending=False, inplace=True)
ml_model_weights_df.head(30)
ml_weights_with_meta_df = ml_model_weights_df.merge(metadata[['object_id', 'description','dimension_text', 'floor']], left_on='feature_name', right_on='object_id', how='left').drop(columns=['object_id'])
for i in range(len(high_abs_corr_df)):
    row = ml_weights_with_meta_df.iloc[i]
    print(f"'{row['feature_name']}',# {row['description']}")

## Correlation and metadata analysis

corr_df = pd.read_csv('5months_3buildings_corrs.csv')
corr_df.shape

In [ ]:
corr_df = shifted_corr_df.copy()

In [ ]:
corr_with_all_meta_df = corr_df.merge(metadata, left_on='sensor_id', right_on='object_id', how='left').drop(columns=['object_id'])
print(len(corr_with_all_meta_df))
corr_with_all_meta_df.drop_duplicates(inplace=True)
print(len(corr_with_all_meta_df))
corr_with_all_meta_df.head()

In [ ]:
corr_with_all_meta_df.dtypes.value_counts()

In [ ]:
from feature_groups import external_weather_measurements

external_corrs = corr_with_all_meta_df[corr_with_all_meta_df['sensor_id'].isin(external_weather_measurements)]
px.bar(external_corrs.sort_values(by='abs_correlation', ascending=False), x='sensor_id', y='correlation', title='Correlation of external weather measurements with target sensor').show()

corr_with_all_meta_df.notna().all(axis=1).idxmax()

In [ ]:
non_numeric_meta_cols = corr_with_all_meta_df.select_dtypes(exclude=['number','bool']).columns.tolist()
len(non_numeric_meta_cols)

In [ ]:
for col in non_numeric_meta_cols:
    unique_values = corr_with_all_meta_df[col].nunique()
    print(f"{col}: {unique_values} unique values")

In [ ]:
corr_with_all_meta_df['device_class'].value_counts()

In [ ]:
corr_with_all_meta_df_wout_target = corr_with_all_meta_df[corr_with_all_meta_df['sensor_id'] != target]
len(corr_with_all_meta_df_wout_target)

We can clearly see here that RC (return control) group has higher max correlation values than FC (fan coil) group.

In [ ]:
for group_col in ['device_class', 'dimension_text', 'building', 'bde_group']:
    grouped = corr_with_all_meta_df_wout_target.groupby(group_col)['abs_correlation']
    mean_corr = grouped.mean().sort_values(ascending=False)
    min_corr = grouped.min().sort_values(ascending=False)
    max_corr = grouped.max().sort_values(ascending=False)
    count = grouped.count().sort_values(ascending=False)
    result_df = pd.DataFrame({'mean_correlation': mean_corr, 'min_correlation': min_corr, 'max_correlation': max_corr, 'count': count})
    print(f"\nTop 10 {group_col} by mean correlation:")
    print(result_df.sort_values(by='mean_correlation', ascending=False))

metadata[metadata['device_class']=='WD']

SP. is setpoint

In [ ]:
corr_with_all_meta_df.sort_values(by='abs_correlation', ascending=False).head(50)[['sensor_id', 'correlation', 'description', 'dimension_text', 'device_class','device','room','bim_floor','bim_room_area']]

What to do with these sensors? By object_id it looks like they are the same but location is different.

In [ ]:
metadata['object_id'].value_counts().value_counts()

In [ ]:
metadata[metadata['object_id']=='B201FC058_4.VT01_1'].T

In [ ]:
corr_with_all_meta_df[corr_with_all_meta_df['room']=='201.A.134']

A Labore (below) is more complex than a corridor (above)

In [ ]:
corr_with_all_meta_df[corr_with_all_meta_df['room']=='201.C.270']

In [ ]:
available_room_info = corr_with_all_meta_df[corr_with_all_meta_df['room'].notnull()]
max_room_corr = available_room_info.groupby('room')['abs_correlation'].max().sort_values(ascending=False)
room_meta = metadata[['room', 'bim_floor', 'bim_room_area', 'bim_room_category', 'bim_energy_category', 'building']].drop_duplicates(subset=['room'])
room_corrs_with_meta = max_room_corr.reset_index().merge(room_meta, on='room', how='left')
room_corrs_with_meta = pd.get_dummies(room_corrs_with_meta, columns=['building'])
room_corrs_with_meta = pd.get_dummies(room_corrs_with_meta, columns=['bim_floor'])
room_corrs_with_meta = pd.get_dummies(room_corrs_with_meta, columns=['bim_room_category'])
room_corrs_with_meta = pd.get_dummies(room_corrs_with_meta, columns=['bim_energy_category'])
room_corrs_with_meta.set_index('room', inplace=True)
room_corrs_with_meta.head(5)

In [ ]:
px.imshow(room_corrs_with_meta.corr(method='spearman'), width=800, height=600, title="Spearman correlation heatmap of room features").show()

STATUS STEP VENTILATION (conversion_index=303.0)

In [ ]:
selected_measurement = corr_with_all_meta_df[corr_with_all_meta_df['conversion_index']==303.0].sort_values('abs_correlation', ascending=False)#status step ventillation

selected_measurement = corr_with_all_meta_df[corr_with_all_meta_df['description'].str.contains('ROOM TEMPERATURE') & (corr_with_all_meta_df['room'].notnull())].copy()

selected_measurement.sort_values('abs_correlation', ascending=False)
selected_measurement['section_id'] = selected_measurement['room'].apply(lambda x: x.split('.')[1])
selected_measurement['section_id'].unique()

In [ ]:
selected_measurement.head(10)

In [ ]:
selected_measurement['bim_room_description'].value_counts()

In [ ]:
import numpy as np

import plotly.express as px

for section_id in selected_measurement['section_id'].unique():
    section_data = selected_measurement[selected_measurement['section_id'] == section_id].copy()
    section_data['room_info'] = section_data['room'].apply(lambda x: x.split('.')[2][1:])
    #section_data = section_data[section_data['device_class'] != 'WC']
    section_data = section_data[section_data['device_class'] != 'FC']
    #section_data = section_data[section_data['device_class'] != 'RC']#we don't have RC sensors for all room
    # correlation matrix
    building_pivot = section_data.pivot_table(index='floor', columns='room_info', values='abs_correlation', aggfunc='max')
    # metadata matrix (room area). use mean in case multiple entries per (floor,room)
    area_pivot = section_data.pivot_table(index='floor', columns='room_info', values='bim_room_area', aggfunc='mean')
    room_descriptions = section_data.pivot_table(index='floor', columns='room_info', values='bim_room_description', aggfunc='first')
    room_energy_categories = section_data.pivot_table(index='floor', columns='room_info', values='bim_energy_category', aggfunc='first')
    room_category = section_data.pivot_table(index='floor', columns='room_info', values='bim_room_category', aggfunc='first')
    building_part = section_data.pivot_table(index='floor', columns='room_info', values='bim_building_part', aggfunc='first')

    # align area_pivot to building_pivot shape
    area_aligned = area_pivot.reindex(index=building_pivot.index, columns=building_pivot.columns)
    description_aligned = room_descriptions.reindex(index=building_pivot.index, columns=building_pivot.columns)
    energy_aligned = room_energy_categories.reindex(index=building_pivot.index, columns=building_pivot.columns)
    category_aligned = room_category.reindex(index=building_pivot.index, columns=building_pivot.columns)
    building_part_aligned = building_part.reindex(index=building_pivot.index, columns=building_pivot.columns)

    z = building_pivot.values
    # create customdata with last dim = 1 so hovertemplate can access customdata[0]
    customdata = np.expand_dims(area_aligned.values, axis=2)
    # combine area and description into a single customdata array (last dim = 2)
    area_vals = area_aligned.values
    desc_vals = description_aligned.values.astype(object)
    energy_vals = energy_aligned.values.astype(object)
    category_vals = category_aligned.values.astype(object)
    building_part_vals = building_part_aligned.values.astype(object)

    customdata = np.empty(area_vals.shape + (5,), dtype=object)
    customdata[..., 0] = area_vals
    customdata[..., 1] = desc_vals
    customdata[..., 2] = energy_vals
    customdata[..., 3] = category_vals
    customdata[..., 4] = building_part_vals

    # Note: update the hovertemplate below to use %{customdata[0]} for area and %{customdata[1]} for description
    # e.g. "Room area: %{customdata[0]:.2f} m²<br>Room description: %{customdata[1]}<extra></extra>"

    fig = px.imshow(
        z,
        x=building_pivot.columns,
        y=building_pivot.index,
        color_continuous_scale='RdBu_r',
        aspect='auto',
        labels={'x': 'room_info', 'y': 'floor', 'color': 'correlation'}
    )

    # attach customdata and set hovertemplate to show bim_room_area
    fig.data[0].customdata = customdata
    fig.data[0].hovertemplate = (
        f"Section: {section_id}<br>"
        "Floor: %{y}<br>"
        "Room: %{x}<br>"
        "Correlation: %{z:.3f}<br>"
        "Room area: %{customdata[0]:.2f} m²<extra></extra><br>"
        "Room description: %{customdata[1]}<extra></extra><br>"
        "Building part: %{customdata[4]}<extra></extra><br>"
        "Room category: %{customdata[3]}<extra></extra><br>"
        "Energy category: %{customdata[2]}<extra></extra><br>"
    )

    fig.update_layout(title=f'Correlation heatmap for status step ventilation sensors in section {section_id}')
    fig.show()

In [ ]:
metadata[(metadata['object_id']=='B201RC268.AM01')]

- What is the difference between: B205WC000.AM01 and B205WC001.AM01 - they have the same description. - Yeah, but they have different device IDs.
- We can extract B205 consumption by taking B205WC000.AM01 - B205WC003.AM01
- Let's make a hiararchical visualization of the sensors with correlation colors...

## Understand high shifted absolute correlations

In [ ]:
selected_building = 'B201'
building_corrs = corr_with_all_meta_df_wout_target[corr_with_all_meta_df_wout_target['building'] == selected_building]
y = building_corrs.pop('abs_correlation')
drop_these = [
    'sensor_id', 'description', 'correlation', 'device', 'room', 'external', 'dimension_index', 'floor',
    'bim_building_name', 'building', 'zone', 'file', 'channel', 'time',
    'bde_recorder', 'bde_object_id',
]
X = building_corrs.drop(columns=drop_these)
X.shape, y.shape

In [ ]:
X['bim_building_part']#what is this?

In [ ]:
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
for col in categorical_cols:
    print(f"{col}: {X[col].nunique()} unique values")

for col in categorical_cols:
    X[col] = X[col].astype('category')#.cat.codes

In [ ]:
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
X.shape

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(n_estimators=30, max_depth=5, learning_rate=0.1, random_state=42)#, enable_categorical=True)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2', n_jobs=-1)
print(f"CV R2: {scores.mean():.4f} ± {scores.std():.4f}")

# finally fit on the training set so model.score on X_test works
model.fit(X_train, y_train)
model.score(X_test, y_test)#r2 on test set

In [ ]:
import shap

explainer = shap.Explainer(model)
shap_values = explainer(X_test)
shap.plots.bar(shap_values, max_display=20)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=20)

In [ ]:
building_corrs['bde_channel_typ'].value_counts()

In [ ]:
building_corrs['device_class'].value_counts()

In [ ]:
building_corrs['bde_object_id'].value_counts()

In [ ]:
building_corrs['class_id'].value_counts()

TODO: what is bde_recorder? is it a hierarchy between sensors how they process or send signals?

building_corrs['bde_recorder'].value_counts()

# Extra ideas

- enable custom ML models (e.g. LightGBM, XGBoost)
    - this approach do not work so far...
    - CHECK: wheather we applied time shift for ML based prediction...
    - if yes: try to understand why correlation analysis works better
- expand correlation analysis to 2024 summer months (same period as test set)
- implement early stopping in model training based on validation set performance
- add training data from 2024

As it turns out:
- feature_selection.py does not use shifted target variable! main.py was correctly implemented!
- main.py never used more features than the original 2 + datetime features?

DONE:
- working holidays and extra weektime sin, cos features added - validation score improved a bit / but not LB score

TODO:
- play with input_seq_len to add more features in main.py

In [ ]:
outputs_dir = 'outputs/'
train_df = pd.read_parquet(os.path.join(outputs_dir, 'preproc_full_train_df.parquet'))
train_df.tail()

In [ ]:
test_df = pd.read_parquet(os.path.join(outputs_dir, 'preproc_test_input_df.parquet'))

## Experimenting with new features

In [ ]:
return_temp_features = metadata[(metadata['building']=='B201') & (metadata['bde_group']=='RC') & (metadata['dimension_index'] == 2.0) & (metadata['conversion_index'] == 118.0)]
return_room_temp = return_temp_features[return_temp_features['description'].str.upper().str.startswith('ROOM TEMPERATURE')].copy()
#return_temp_features[['object_id', 'description', 'device_class', 'channel', 'class_id', 'device', 'room', 'bde_recorder', 'bde_object_id']]
return_room_temp

In [ ]:
return_room_temp = return_room_temp.merge(corr_df, left_on='object_id', right_on='sensor_id', how='left').drop(columns=['sensor_id'])
return_room_temp.head()

In [ ]:
agg_room_temp = return_room_temp.groupby('room').agg({
    'object_id':'count', 'bim_room_area':'max',
    'bim_room_area': 'max',
    'abs_correlation':'mean', 'bim_room_description':'first', 'bim_room_category':'first'
    }).sort_values(by='object_id', ascending=False).rename(columns={'object_id':'num_sensors'}).reset_index()
agg_room_temp.head()

In [ ]:
agg_room_temp['size'] = agg_room_temp['num_sensors']#*2+3
px.scatter(agg_room_temp, x='abs_correlation', y='bim_room_area', color='bim_room_description', size='size',
           hover_data=['bim_room_description','room'],
           log_x=False, log_y=True,
           title='Number of room temperature sensors vs room area').show()

In [ ]:
return_room_temp[return_room_temp['bim_room_description']=='Dining']['abs_correlation'].hist(bins=10)

In [ ]:
from feature_groups import aggregate_sensor_values

return_room_temp['bim_room_description'] = return_room_temp['bim_room_description'].fillna('NoDescription')
return_room_temp['bim_room_category'] = return_room_temp['bim_room_category'].fillna('NoCategory')
return_room_temp['bim_energy_category'] = return_room_temp['bim_energy_category'].fillna('NoEnergyCategory')

temp_by_room_type = {}
for col in ['bim_room_description', 'bim_energy_category']:#, 'bim_room_category']:
    print(f"{col}: {return_room_temp[col].nunique()} unique values")
    room_meta = return_room_temp[col].unique()
    for description in room_meta:
        desc_metadata = return_room_temp[return_room_temp[col] == description]
        weighted_temp = aggregate_sensor_values(desc_metadata, train_df, key_col='room', weight_col='bim_room_area')# avg room temp weighted by room area
        temp_by_room_type[description] = weighted_temp

temp_by_room_type['AllRooms'] = aggregate_sensor_values(return_room_temp, train_df, key_col='room', weight_col='bim_room_area')# avg room temp weighted by room area

In [ ]:
len(return_room_temp), len(temp_by_room_type)

In [ ]:
temp_by_room_description_df = pd.DataFrame(temp_by_room_type)
temp_by_room_description_df['target'] = train_df[target].shift(-18)

In [ ]:
temp_by_room_description_df.corr(method='spearman').sort_values(by='target', ascending=False)['target'].drop('target').plot.bar(figsize=(15,5), title='Spearman correlation of weighted room temperatures with target sensor')

In [ ]:
melted_temp_df = temp_by_room_description_df.reset_index().melt(id_vars=['time'], var_name='room_description', value_name='weighted_room_temp')
px.line(melted_temp_df, x='time', y='weighted_room_temp', color='room_description', title='Weighted room temperatures by room description over time').show()


agg_room_temp['size'] = agg_room_temp['num_sensors']#*2+3
px.scatter(agg_room_temp, x='abs_correlation', y='bim_room_area', color='bim_room_category', size='size',
           hover_data=['bim_room_description'],
           log_x=False, log_y=True,
           title='Number of room temperature sensors vs room area').show()

In [ ]:
dict(zip(return_temp_features['dimension_index'], return_temp_features['dimension_text']))

In [ ]:
stop

In [ ]:
submission_sample = pd.read_csv('submissions/fc_main/abs_spcorr0.3_submission_file.csv')
submission_sample_values = submission_sample['TARGET_VARIABLE'].values
submission_sample.rename(columns={'ID':'time', 'TARGET_VARIABLE':target}, inplace=True)
submission_sample.head()

In [ ]:
import numpy as np
from utils import HungarianWorkdayAnalyzer

workday_analyzer = HungarianWorkdayAnalyzer()

def add_more_features(X, feature_names, lags=0, agg_windows=['1h', '6h', '24h'], target=target):
    
    if lags > 0:
        for feature in feature_names:
            for lag in range(1, lags+1):
                X[f'{feature}_lag_{lag}'] = X[feature].shift(lag)

    if target in X.columns:
        for window in agg_windows:
            X[f'{target}_rolling_mean_{window}'] = X[target].rolling(window=window).mean()
            X[f'{target}_rolling_std_{window}'] = X[target].rolling(window=window).std()
            X[f'{target}_rolling_quantiile_0.90_{window}'] = X[target].rolling(window=window).quantile(0.9)
            X[f'{target}_rolling_quantiile_0.10_{window}'] = X[target].rolling(window=window).quantile(0.1)
    
    X.reset_index(inplace=True)
    X['hour'] = X['time'].dt.hour
    X['weekday'] = X['time'].dt.weekday
    X['hour_sin'] = np.sin(2 * np.pi * X['hour'] / 24)
    X['hour_cos'] = np.cos(2 * np.pi * X['hour'] / 24)
    X['weekday_sin'] = np.sin(2 * np.pi * X['weekday'] / 7)
    X['weekday_cos'] = np.cos(2 * np.pi * X['weekday'] / 7)

    X['is_holiday'] = X['time'].apply(workday_analyzer.is_official_holiday)
    X['is_weekend'] = X['time'].apply(workday_analyzer.is_weekend)
    X['is_working_day'] = X['time'].apply(workday_analyzer.is_working_day)

    X.set_index('time', inplace=True)

    
    return X

In [ ]:
correlation_results_fp = 'b205_sensor_correlations_shift-180min.csv'
correlation_df = pd.read_csv(correlation_results_fp)
correlation_df.head()

In [ ]:
extra_features = []#correlation_df[ correlation_df['abs_correlation'] > 0.4 ]['sensor_id'].tolist()

In [ ]:
lags = 6
agg_windows=['1h', '6h', '24h']
active_features = [
    target,#we only use it historically
    "B205WC000.AM01",
    "B106WS01.AM54",
]
active_features += extra_features
active_features = list(set(active_features))
print(f"Number of active features: {len(active_features)}")
prediction_horizon = 6*3  # 3 hours ahead, 6 samples per hour
small_train_df = train_df#.tail(6*24*7*12+prediction_horizon)
y = small_train_df[target]
X = small_train_df[active_features].shift(prediction_horizon)
mask = y.notnull()
for i in range(prediction_horizon):
    mask.iloc[i] = False
X = X[mask]
y = y[mask]
X = add_more_features(X, active_features, lags=6, agg_windows=agg_windows, target=target)

#missing_feature_ratios = X.isnull().mean().sort_values(ascending=False)
#missing_threshold = 0.1
#missing_feature_ratios = missing_feature_ratios[missing_feature_ratios > missing_threshold]
#X.drop(columns=missing_feature_ratios.index, inplace=True)

- Models are not good enough to predict target variable from among 5K+ sensors
- some feature selection is needed!
   - can we do some knowledge network based feature selection?
   - or simply select by sensor name? Can LNNs help here?
   - we should also build some cause-effect graph from data? if sensors are highly correlated in time, let's select the root cause sensors only?
   - what about reinforcement learning based feature selection?
      - it gets the knowledge graph as input
      - tries to learn some mask over the sensors
      - reward is based on validation score of the model trained on selected features

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

model_id = 'rf'

if model_id == 'lr':
    model_item = LinearRegression()
elif model_id == 'lasso':
    model_item = Lasso(alpha=0.01)
elif model_id == 'rf':
    #too bad: overfits
    model_item = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42, n_jobs=10)

# simple pipeline: mean imputation + linear regression
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ('scaler', StandardScaler()),
    (model_id, model_item),
])

# time-series aware CV
tscv = TimeSeriesSplit(n_splits=5)

# cross-validate using negative MSE (convert to RMSE for reporting) and R2
mse_scores = cross_val_score(pipeline, X, y, scoring="neg_mean_squared_error", cv=tscv, n_jobs=-1)
rmse_scores = np.sqrt(-mse_scores)
r2_scores = cross_val_score(pipeline, X, y, scoring="r2", cv=tscv, n_jobs=-1)

print(f"CV RMSE: mean={rmse_scores.mean():.4f}, std={rmse_scores.std():.4f}")
print(f"CV R2  : mean={r2_scores.mean():.4f}, std={r2_scores.std():.4f}")

# fit final model on all available training data
pipeline.fit(X, y)
lr_model = pipeline  # fitted pipeline

weights_df = pd.DataFrame({
    'feature': X.columns,
    'weight': lr_model.named_steps[model_id].coef_ if model_id in ['lr', 'lasso'] else lr_model.named_steps[model_id].feature_importances_
})
weights_df['abs_weight'] = weights_df['weight'].abs()
weights_df = weights_df[weights_df['abs_weight'] > 0]
weights_df.sort_values('abs_weight', ascending=False, inplace=True)
weights_df.reset_index(drop=True, inplace=True)
weights_df.head(20)

In [ ]:
import shap

transformed_X = pipeline.named_steps['imputer'].transform(X)
transformed_X = pipeline.named_steps['scaler'].transform(transformed_X)
explainer = shap.Explainer(pipeline.named_steps[model_id], transformed_X)
shap_values = explainer(transformed_X)

In [ ]:
shap.summary_plot(shap_values, features=transformed_X, feature_names=X.columns)

In [ ]:
submission_ts_str = submission_sample['time'].tolist()
X_test = test_df[active_features].copy()
X_test = add_more_features(X_test, active_features, lags=lags, agg_windows=agg_windows, target=target)
X_test = X_test[X.columns]
X_test['time_str'] = X_test.index.strftime('%Y-%m-%d_%H:%M:%S')
X_test = X_test[X_test['time_str'].isin(submission_ts_str)].drop(columns=['time_str'])
assert len(X_test) == len(submission_sample)
X_test = pipeline.named_steps['imputer'].transform(X_test)
X_test = pipeline.named_steps['scaler'].transform(X_test)
test_predictions = pipeline.named_steps[model_id].predict(X_test)

last_known_target = train_df[target].rolling(6*24).mean().iloc[-(prediction_horizon+1)]
first_prediction = test_predictions[0]
difference = first_prediction - last_known_target
print(f"Last known target: {last_known_target:.4f}", f"First prediction: {first_prediction:.4f}", f"Difference: {difference:.4f}")

px.line(test_df[target], title='Test set target variable over time')

In [ ]:
new_submission = submission_sample.copy()
new_submission[target] = test_predictions + 0.5#difference
new_submission.columns = ['ID', 'TARGET_VARIABLE']

In [ ]:
fig = px.line(new_submission, x='ID', y='TARGET_VARIABLE', title='Submission Predictions')
fig.add_scatter(x=new_submission['ID'], y=submission_sample_values, mode='lines', name='submission_sample_values')
fig

In [ ]:
new_submission.to_csv('new_submission.csv', index=False)

shap.plots.beeswarm(shap_values)

In [ ]:
predictors = [
    "B205WC000.AM01",  # a supply temperature chilled water
    "B106WS01.AM54",  # an external temperature
    #"B201AH163.AC63",
    #"B205WC140.AC21"
]
shift_target = False

In [ ]:
means = train_df[predictors + [target]].mean()
stds = train_df[predictors + [target]].std()

In [ ]:
k = 6*24*7*12
series = train_df.head(k)[predictors+[target]].copy()
series = (series - means) / stds
series = series.reset_index()
if shift_target:
    series[target] = series[target].shift(-6*3)
series['weekday'] = series['time'].apply(lambda x: x.weekday())
series_melted = series.melt(id_vars='time')
series_melted.head()

In [ ]:
import plotly.express as px

- "B205WC000.AM01" is not a good predictor: 3 hour shifted target spike is earlier than predictor spike
- around holidays (non working days) behavior is caotic
- also around periods with missing sensor data
- the changes in the shape of "B205WC000.AM01" (spike, steps, constant) give you the magnitude of the target
- in the end we must predict missing sensor data when they are missing during train or test time - NOT to mislead the model!
- from 

In [ ]:
px.line(series_melted, x='time', y='value', color='variable')

In [ ]:
series_tail = train_df.tail(k)[predictors+[target]].copy()
series_tail = (series_tail - means) / stds
series_tail = series_tail.reset_index()
if shift_target:
    series_tail[target] = series_tail[target].shift(-6*3)
series_tail['weekday'] = series_tail['time'].apply(lambda x: x.weekday())
series_melted = series_tail.melt(id_vars='time')
px.line(series_melted, x='time', y='value', color='variable')

In [ ]:
series_test = test_df[predictors+[target]].copy().reset_index()
series_test['time_str'] = series_test['time'].dt.strftime('%Y-%m-%d_%H:%M:%S')
series_test = series_test.merge(submission_sample, left_on='time_str', how='left', right_on='time', suffixes=('', '_pred')).drop(columns=['time_str', 'time_pred'])
series_test.set_index('time', inplace=True)
series_test = (series_test - means) / stds
series_test = series_test.reset_index()
series_melted = series_test.melt(id_vars='time')
px.line(series_melted, x='time', y='value', color='variable')

# Feature standard deviations

In [ ]:
test_path = 'data/splits/train_2025-04_to_2025-05_test_2025-06_to_2025-07'

In [ ]:
test_df = pd.read_parquet(os.path.join(test_path, 'preproc_test_input_df.parquet'))
test_df.shape

In [ ]:
test_df.head()

In [ ]:
missing_ratios = test_df.isnull().mean().sort_values(ascending=False)

In [ ]:
missing_ratios.head(500).plot.line(figsize=(15,5), title='Top features with highest missing ratios in test set')

In [ ]:
nunique_counts = test_df.nunique().sort_values(ascending=True)
nunique_counts.head(3000).plot.line(figsize=(15,5), title='Top features with highest unique value counts in test set')

In [ ]:
nunique_counts

In [ ]:
feature_information = pd.DataFrame({
    'missing_ratio': missing_ratios,
    'nunique_count': nunique_counts,
})
feature_information.head()

In [ ]:
potential_features = feature_information[
    (feature_information['missing_ratio'] < 0.2) &
    (feature_information['nunique_count'] > 2)
].copy()
print(f"Number of potential features: {len(potential_features)}")

In [ ]:
feature_information.to_csv('test_set_feature_information.csv')

## Splits refactored correlations

In [ ]:
corr_dir = 'correlation_analysis_shifted_test_filtered'
corr_summary = pd.read_csv(os.path.join(corr_dir, 'correlation_summary_across_splits.csv'))
corr_summary.head()

In [ ]:
corr_summary['sensor_type'] = corr_summary['sensor_id'].apply(lambda x: x.split('.')[1].split('_')[0] if '_' in x.split('.')[1] else x.split('.')[1])
corr_summary['sensor_type'].value_counts().head(20)

In [ ]:
corr_summary.groupby('sensor_type')['min_abs_correlation'].mean().sort_values(ascending=False).head(20)

In [ ]:
sensor_type_info = pd.DataFrame({
    'mean_min_abs_correlation': corr_summary.groupby('sensor_type')['min_abs_correlation'].mean(),
    'count': corr_summary['sensor_type'].value_counts()
})
px.scatter(sensor_type_info, x='count', y='mean_min_abs_correlation', text=sensor_type_info.index, title='Sensor Type Correlation Analysis', log_x=True).show()

### Compare previous predictions

In [17]:
pred1 = pd.read_csv('submissions/origi_github/really_from_scratch.csv')
pred2 = pd.read_csv('submissions/origi_github/annual_time_features_removed.csv')
pred3 = pd.read_csv('submissions/origi_github/17_cooler_valves_normalized.csv')
pred4 = pd.read_csv('submissions/origi_github/multichannel_17AC21.csv')
pred5 = pd.read_csv('submissions/origi_github/multichannel_allAC21.csv')
pred6 = pd.read_csv('submissions/origi_github/multichannel_128origi2_8AC21.csv')
pred7 = pd.read_csv('submissions/origi_github/mc_VT03_2_64dim_origi128dim.csv')
pred8 = pd.read_csv('submissions/origi_github/mc_allrooms_VT03_2_64dim_origi128.csv')
pred9 = pd.read_csv('submissions/origi_github/mc_origi128_coolerv64_activesp64_co2_64.csv')
pred10 = pd.read_csv('submissions/origi_github/mc_origi128_coolerv64_activesp64_co2_64_humidity64.csv')
pred11 = pd.read_csv('submissions/origi_github/mc_origi128_coolerv64_activesp64_co2_64_humidity_64_noroomwiseavg.csv')
pred12 = pd.read_csv('submissions/origi_github/mc_AM22_added_to_CO2.csv')
pred13 = pd.read_csv('submissions/origi_github/mc_origi_coolerv_activesp_co2_controller.csv')
pred14 = pd.read_csv('submissions/origi_github/humidity_is_back_32dim.csv')
pred15 = pd.read_csv('submissions/origi_github/mc_origi2_ac21_vt03_2.csv')
pred16 = pd.read_csv('submissions/origi_github/mc_coolerv_fcroomtemp.csv')
pred17 = pd.read_csv('outputs_offline_channelexp_2025/useCoolerV_True_useActiveSp_False_useCO2_False_useHumidity_False_useCtrlBldg_False_useFCRoomT_True_useRCRoomT_False_extraInfo_AM11/submission.csv')
pred18 = pd.read_csv('outputs_offline_channelexp_2025/useCoolerV_True_useActiveSp_False_useCO2_False_useHumidity_False_useCtrlBldg_False_useFCRoomT_True_useRCRoomT_False_extraInfo_AM02/submission.csv')
pred19 = pd.read_csv('outputs_offline_channelexp_2025/useCoolerV_True_useActiveSp_False_useCO2_False_useHumidity_False_useCtrlBldg_False_useFCRoomT_True_useRCRoomT_False_extraInfo_AM31/submission.csv')
pred20 = pd.read_csv('outputs_offline_channelexp_2025/useCoolerV_True_useActiveSp_False_useCO2_False_useHumidity_False_useCtrlBldg_False_useFCRoomT_True_useRCRoomT_False_extraInfo_AM02_AM22/submission.csv')
pred21 = pd.read_csv('outputs_offline_channelexp_2025/useCoolerV_True_useActiveSp_False_useCO2_False_useHumidity_False_useCtrlBldg_False_useFCRoomT_True_useRCRoomT_False_extraInfo_AM31_AM02/submission.csv')

In [18]:
merged_preds = pred1.merge(pred2, on='ID', suffixes=('_scratch', '_no_annual'))
merged_preds = merged_preds.merge(pred3, on='ID')
merged_preds.columns = ['ID', 'TARGET_VARIABLE_scratch', 'TARGET_VARIABLE_no_annual', 'TARGET_VARIABLE_cooler_valves']
merged_preds = merged_preds.merge(pred4, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_multichannel'}, inplace=True)
merged_preds = merged_preds.merge(pred5, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_allAC21'}, inplace=True)
merged_preds = merged_preds.merge(pred6, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_128origi2_8AC21'}, inplace=True)
merged_preds = merged_preds.merge(pred7, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_VT03_2'}, inplace=True)
merged_preds = merged_preds.merge(pred8, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_allrooms_VT03_2'}, inplace=True)
merged_preds = merged_preds.merge(pred9, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64'}, inplace=True)
merged_preds = merged_preds.merge(pred10, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64_humidity64'}, inplace=True)
merged_preds = merged_preds.merge(pred11, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64_humidity_64_noroomwiseavg'}, inplace=True)
merged_preds = merged_preds.merge(pred12, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_AM22_added_to_CO2'}, inplace=True)
merged_preds = merged_preds.merge(pred13, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi_coolerv_activesp_co2_controller'}, inplace=True)
merged_preds = merged_preds.merge(pred14, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_humidity_is_back_32dim'}, inplace=True)
merged_preds = merged_preds.merge(pred15, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi2_ac21_vt03_2'}, inplace=True)
merged_preds = merged_preds.merge(pred16, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_coolerv_fcroomtemp'}, inplace=True)
merged_preds = merged_preds.merge(pred17, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM11'}, inplace=True)
merged_preds = merged_preds.merge(pred18, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM02'}, inplace=True)
merged_preds = merged_preds.merge(pred19, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM31'}, inplace=True)
merged_preds = merged_preds.merge(pred20, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM02_AM22'}, inplace=True)
merged_preds = merged_preds.merge(pred21, on='ID')
merged_preds.rename(columns={'TARGET_VARIABLE':'TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM31_AM02'}, inplace=True)
merged_preds.head()

,ID,TARGET_VARIABLE_scratch,TARGET_VARIABLE_no_annual,TARGET_VARIABLE_cooler_valves,TARGET_VARIABLE_multichannel,TARGET_VARIABLE_allAC21,TARGET_VARIABLE_128origi2_8AC21,TARGET_VARIABLE_mc_VT03_2,TARGET_VARIABLE_mc_allrooms_VT03_2,TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64,...,TARGET_VARIABLE_mc_AM22_added_to_CO2,TARGET_VARIABLE_mc_origi_coolerv_activesp_co2_controller,TARGET_VARIABLE_humidity_is_back_32dim,TARGET_VARIABLE_mc_origi2_ac21_vt03_2,TARGET_VARIABLE_mc_coolerv_fcroomtemp,TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM11,TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM02,TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM31,TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM02_AM22,TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM31_AM02
0,2025-06-01_00:00:00,10.492555,10.027039,10.252860,10.022106,10.288072,9.855820,10.962930,10.379384,10.336004,...,9.710895,10.077850,10.409308,11.330806,11.305099,11.120352,11.026722,11.161301,10.602311,10.799511
1,2025-06-01_00:10:00,10.522740,10.068272,10.241058,10.004525,10.281844,9.838559,10.967688,10.391337,10.329133,...,9.712272,10.079725,10.413029,11.320572,11.308064,11.131010,11.032549,11.169525,10.617745,10.790894
2,2025-06-01_00:20:00,10.514061,10.075539,10.185513,10.044414,10.293754,9.857683,10.966682,10.391874,10.344495,...,9.716020,10.087336,10.410601,11.327698,11.296410,11.138931,11.024201,11.133351,10.610960,10.780664
3,2025-06-01_00:30:00,10.523720,10.060943,10.175034,10.078127,10.306415,9.873923,10.977646,10.400737,10.332720,...,9.718886,10.064000,10.412435,11.334863,11.300497,11.133252,11.021111,11.138498,10.619396,10.781901
4,2025-06-01_00:40:00,10.537749,10.070622,10.212947,10.094210,10.305692,9.876386,10.983428,10.421968,10.354805,...,9.725398,10.068003,10.422798,11.343650,11.304921,11.134689,11.029551,11.152328,10.622685,10.791796


In [6]:
merged_preds['combined_allAC21'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_allAC21']

In [7]:
merged_preds['combined_ac21_vt03_2'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi2_ac21_vt03_2']
missed_submission = merged_preds[['ID', 'combined_ac21_vt03_2']].copy()
missed_submission.rename(columns={'combined_ac21_vt03_2':'TARGET_VARIABLE'}, inplace=True)
missed_submission.to_csv('missed_submission_ac21_vt03_2.csv', index=False)  

In [8]:
merged_preds['combined_fcroomtemp'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_coolerv_fcroomtemp']
final_submission = merged_preds[['ID', 'combined_fcroomtemp']].copy()
final_submission.rename(columns={'combined_fcroomtemp':'TARGET_VARIABLE'}, inplace=True)
final_submission.to_csv('combined_08_02_fcroomtemp.csv', index=False)

In [9]:
merged_preds['combined_am11'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM11']
final_submission_am11 = merged_preds[['ID', 'combined_am11']].copy()
final_submission_am11.rename(columns={'combined_am11':'TARGET_VARIABLE'}, inplace=True)
final_submission_am11.to_csv('combined_08_02_am11.csv', index=False)

In [10]:
merged_preds['combined_am02'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM02']
final_submission_am02 = merged_preds[['ID', 'combined_am02']].copy()
final_submission_am02.rename(columns={'combined_am02':'TARGET_VARIABLE'}, inplace=True)
final_submission_am02.to_csv('combined_08_02_am02.csv', index=False)

In [20]:
merged_preds['combined_egal_am02'] = 0.5*merged_preds['TARGET_VARIABLE_no_annual'] + 0.5*merged_preds['TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM02']
final_submission_am02 = merged_preds[['ID', 'combined_egal_am02']].copy()
final_submission_am02.rename(columns={'combined_egal_am02':'TARGET_VARIABLE'}, inplace=True)
final_submission_am02.to_csv('combined_05_05_am02.csv', index=False)

In [13]:
merged_preds['combined_am31'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM31']
final_submission_am31 = merged_preds[['ID', 'combined_am31']].copy()
final_submission_am31.rename(columns={'combined_am31':'TARGET_VARIABLE'}, inplace=True)
final_submission_am31.to_csv('combined_08_02_am31.csv', index=False)

In [16]:
merged_preds['combined_am02_am22'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM02_AM22']
final_submission_am02_am22 = merged_preds[['ID', 'combined_am02_am22']].copy()
final_submission_am02_am22.rename(columns={'combined_am02_am22':'TARGET_VARIABLE'}, inplace=True)
final_submission_am02_am22.to_csv('combined_08_02_am02_am22.csv', index=False)

In [19]:
merged_preds['combined_am02_am31'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi2_ac21_fctemp_AM31_AM02']
final_submission_am02_am31 = merged_preds[['ID', 'combined_am02_am31']].copy()
final_submission_am02_am31.rename(columns={'combined_am02_am31':'TARGET_VARIABLE'}, inplace=True)
final_submission_am02_am31.to_csv('combined_08_02_am31_am02.csv', index=False)

merged_preds['VT03_2_combined'] = 0.6*merged_preds['TARGET_VARIABLE_no_annual'] + 0.4*merged_preds['TARGET_VARIABLE_mc_allrooms_VT03_2']
vt03_2_submission = merged_preds[['ID', 'VT03_2_combined']].rename(columns={'VT03_2_combined':'TARGET_VARIABLE'})
vt03_2_submission.to_csv('vt03_2_combined_06_04_submission.csv', index=False)

merged_preds['VT03_01_AC21_02_origi_07'] = 0.1*merged_preds['TARGET_VARIABLE_mc_VT03_2'] + 0.2*merged_preds['TARGET_VARIABLE_allAC21'] + 0.7*merged_preds['TARGET_VARIABLE_no_annual']
vt03_2_ac21_origi_submission = merged_preds[['ID', 'VT03_01_AC21_02_origi_07']].rename(columns={'VT03_01_AC21_02_origi_07':'TARGET_VARIABLE'})
vt03_2_ac21_origi_submission.to_csv('vt03_02_ac21_01_origi_07_submission.csv', index=False)

merged_preds['combined_3_extra_channels'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64']
combined_submission_3 = merged_preds[['ID', 'combined_3_extra_channels']].rename(columns={'combined_3_extra_channels':'TARGET_VARIABLE'})
combined_submission_3.to_csv('combined_08_02_3extrachannels_submission.csv', index=False)

merged_preds['combined_3_extra_channels_with_AM22'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_AM22_added_to_CO2']
combined_submission_3 = merged_preds[['ID', 'combined_3_extra_channels_with_AM22']].rename(columns={'combined_3_extra_channels_with_AM22':'TARGET_VARIABLE'})
combined_submission_3.to_csv('combined_08_02_3extrachannels_with_AM22_submission.csv', index=False)

merged_preds['combined_humidity'] = 0.9*merged_preds['TARGET_VARIABLE_no_annual'] + 0.1*merged_preds['TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64_humidity64']
combined_submission_humidity = merged_preds[['ID', 'combined_humidity']].rename(columns={'combined_humidity':'TARGET_VARIABLE'})
combined_submission_humidity.to_csv('combined_09_01_humidity_submission.csv', index=False)

In [ ]:
merged_preds['combined_humidity_noroomavg'] = 0.7*merged_preds['TARGET_VARIABLE_no_annual'] + 0.3*merged_preds['TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64_humidity_64_noroomwiseavg']
combined_submission_humidity_noroomavg = merged_preds[['ID', 'combined_humidity_noroomavg']].rename(columns={'combined_humidity_noroomavg':'TARGET_VARIABLE'})
combined_submission_humidity_noroomavg.to_csv('combined_07_03_humidity_noroomavg_submission.csv', index=False)

In [ ]:
merged_preds['combined_with_controller'] = 0.8*merged_preds['TARGET_VARIABLE_no_annual'] + 0.2*merged_preds['TARGET_VARIABLE_mc_origi_coolerv_activesp_co2_controller']
combined_submission = merged_preds[['ID', 'combined_with_controller']].copy()
combined_submission.rename(columns={'combined_with_controller':'TARGET_VARIABLE'}, inplace=True)
combined_submission.to_csv('combined_08_02_with_controller_submission.csv', index=False)

In [ ]:
merged_preds['combi_humidity_back_32dim'] = 0.6*merged_preds['TARGET_VARIABLE_no_annual'] + 0.4 *merged_preds['TARGET_VARIABLE_humidity_is_back_32dim']
combined_submission_humidity_32dim = merged_preds[['ID', 'combi_humidity_back_32dim']].rename(columns={'combi_humidity_back_32dim':'TARGET_VARIABLE'})
combined_submission_humidity_32dim.to_csv('combined_06_04_humidity_back_32dim_submission.csv', index=False)  

In [ ]:
merged_melted = merged_preds.melt(id_vars='ID', value_vars=[
    #'TARGET_VARIABLE_scratch',
    'TARGET_VARIABLE_no_annual',
    'TARGET_VARIABLE_cooler_valves',
    'TARGET_VARIABLE_mc_allrooms_VT03_2',
    'TARGET_VARIABLE_mc_origi128_coolerv64_activesp64_co2_64',
    #'TARGET_VARIABLE_multichannel',
    #'TARGET_VARIABLE_allAC21',
    #'TARGET_VARIABLE_128origi2_8AC21',
    #'combined_allAC21',
    #'TARGET_VARIABLE_mc_allrooms_VT03_2',
    #'VT03_2_combined',
    #'combined_3_extra_channels',
    #'combined_humidity',
    #'combined_humidity_noroomavg'
    #'combined_with_controller',
], var_name='model', value_name='prediction')
px.line(merged_melted, x='ID', y='prediction', color='model', title='Comparison of Predictions from Two Models').show()

# What channels should I include?

In [ ]:
test_filtered_corrs = pd.read_csv(os.path.join('correlation_analysis_shifted_test_filtered', 'correlation_summary_across_splits.csv'))
test_filtered_corrs.head()

In [ ]:
print(len(metadata))
metadata_sorted = metadata.sort_values(by=['object_id', 'bim_room_area'], ascending=[True, False]).drop_duplicates(subset=['object_id'])
len(metadata_sorted)

In [ ]:
test_filtered_corrs_with_meta = test_filtered_corrs.merge(metadata_sorted, left_on='sensor_id', right_on='object_id', how='left').drop(columns=['object_id'])
print(len(test_filtered_corrs_with_meta))

In [ ]:
used_channels = [
    'AC21',  # cooler valve
    'VT03_2',  # active setpoints
    'AM21',  # CO2 sensors
]
used_channels += ['AM45', 'AM45_1', 'AM51']#humidity sensors

Currently used sensors

In [ ]:
test_filtered_corrs_with_meta[test_filtered_corrs_with_meta['channel'].isin(used_channels)].shape

In [ ]:
not_used_sensors = test_filtered_corrs_with_meta[~test_filtered_corrs_with_meta['channel'].isin(used_channels)]
not_used_sensors = not_used_sensors.sort_values(by='mean_abs_correlation', ascending=False)
not_used_sensors.head(20)

There was high missing room info for humidity sensors

humidity_sensors['room'].value_counts().value_counts()

humidity_sensors['room'].isnull().sum()

humidity_sensors[humidity_sensors['room'].isnull()]

humidity_sensors['description'].apply(lambda x: ' '.join(x.split()[:-1])).value_counts()

We have 61 sensors for the control building B205
- we should add it as a separate channel group

In [ ]:
not_used_sensors = not_used_sensors[not_used_sensors['sensor_id'].str.startswith('B205')]
not_used_sensors.shape

- AM15,  # return air temperature
- AM71,  # setpoint temperature (BUT Kelvin) - should we add it to active sp channel group? NO but use those that are related to the control building B205
- AM45, AM45_1,  # air humidity + AM51 room humidity

In [ ]:
channels_stats = not_used_sensors.groupby('channel')['mean_abs_correlation'].agg(['mean', 'count'])
channels_stats.sort_values(by='mean', ascending=False).head(20)

In [ ]:
channels_stats.sort_values(by='count', ascending=False).head(15)